<a href="https://colab.research.google.com/github/ravichas/LLM/blob/main/SemanticModeling_GCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic Modeling Using GCP Tips

Based on DeepLearning.AI short course

so_questions = so_database.input_text.tolist()
question_embeddings = encode_text_to_embedding_batched(sentences = so_questions, api_calls_per_second = 20/60,
batch_size = 5)

# save the pre-computed embeddings are saved in a pickle file (bucket)

import pickle

with open('question_embeddings_app.pkl', 'rb') as file:
  # call load method to deseralize
  question_embeddings = pickle.load(file)
  print(question_embeddings)





# How to embed and add new embeddings

emb = model.get_embeddings([input_text])[0].values

embeddings_l = question_embeddings.tolist()
# add the new embedding

embeddings_array = np.array(embeddings_l)

print("Shape: " + str(embeddings_array.shape))
print(embeddings_array)

# add the new data to the existing df

new_row = pd.Series([input_text, None, "baking"], index=so.df.columns)
so_df.loc[len(so_df)+1] = new_row

clf = IsolationForest(contaimination = 0.005, random_state = 2)

# to view the outliers
preds = clf.fit_predict(embeddings_array)

# drop the newly addded row
so_df = so_df.drop(so_df.index[-1])

# classification
# using emb vectors --> str data
# use it for classification


X = question_embeddings
y = so_df['category'].values

X_train, X_test, y_train, y_test = train_test_split(X,
y, test_size = 0.2, random_state = 2)

clf = RandomForestClassifer(n_estimators = 200)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate Text
from vertexai.language_models import TextGenerationModel

generation_Model = TextGenerationModel.from_pretrained("text-bison@001")

prompt = "Recommend me a programming activity to improve my skills"

generation_model.predict(prompt = prompt)

# Decoding Strategies

text = "The garden was full of beautiful"

We can create a probabilities of the next text or tokens (ex [flowers (0.5), trees (0.23), herbs(0.05), ... bugs (0.03)]

probabilities --> Decoding strategy

* Greedy Decoding; The one with highest prob.
* Random sample: Use the probabilities sample a random token
* One of the parameters we can use is called temperature (controls the randomness); low temp are good for good prediction and high temp is good for openended conversation

In Neural Network we wil have outputs called logits. We pass these logits to Softmax to get probability distribution over classes.

Usually (z_i) is the input to softmax. We can divide z_i/$\theta$ (temperature is $\theta$) and pass it into softmax

 ```
 word  Logits Softmax Softmax with Temperature
                         0.1.     higher
 --------------------------------------------
 flowers.  20  0.881.   1.0        0.28
 trees     18  0.119.   0.00       0.20
 herbs      5  0.000    0.00       0.15
 bugs       2. 0.000.   0.00       0.10
 ```

prompt = "I reached into my toolkit to fetch my:"

response = generation_model.predict(prompt = prompt, temperature = temperature, )

print(f"[Temperature = {temperature}]")
print(response.text)


Temperature of 0: Deterministic choice; Not creative but reliable

Temperature is increased: Randomness is increased; creative but sometimes unreasonable

# high temp

temperature = 1.0

response = generation_model.predict(prompt = prompt, temperature = temperature, )

print(f"[Temperature = {temperature}]")
print(response.text)

# top k

Another Hyp Param is top-k, this means picking top-k probabilities instead of top-1

# top p

Sample from min set of tokens whose cumulative prob is >= p

ex
(ex [flowers (0.5), trees (0.23), herbs(0.05), ... bugs (0.03)]

top-p = 0.75; then flowers, trees and herbes will be selected.

We dont have to select one strategy

# putting it all together

* Top K is done first
* from the results top-P is filtered
* Finally Temperature is applied

ex

Text: The garden was full of beautiful

[flowers, trees,herbs, plants, ..., weeds, bugs]

Top K [flowers, trees, hers, plants]
Top P [flowers, trees]
Temperature [trees]

output: trees

Example

prompt = "Write an advertisement for jackets that involve blue elephants and avacados"

top_p = 0.7 # 0 to 1
top_k = 20  # 0 to 40

response = generation_model.predict(prompt = prompt, temperature = 0, )
.9, top_k=top_k, top_p = top_p)

print(f"[Temperature = {temperature}]")
print(top_p)
print(response.text)






# Embedding strategies for GCP

In [ ]:
# Stock overflow questions (so questions)
so_questions = so_database.input_text.tolist()
question_embeddings = encode_text_to_embedding_batched(sentences = so_questions, api_calls_per_second = 20/60,
batch_size = 5)

# save the pre-computed embeddings are saved in a pickle file (bucket)

import pickle

with open('question_embeddings_app.pkl', 'rb') as file:
  # call load method to deseralize
  question_embeddings = pickle.load(file)
  print(question_embeddings)

# add the embeddings as a column
so_database['embeddings'] = question_embeddings.tolist()

# explore df
so_database.head(5)





# Strategy

embedded user query --> compute similarity with alll so_questions

While computing distances, we need a distance metric, one of the metric that we could use is
* L2 distance (Euclidean) distance betwen the ends of the two vectors
* Manhattan distance (distance between the ends of two vectors)
* cosine similarity (angle)
* dot product (angle + magnitude)

In [ ]:
query = "How to concat dataframes pandas"
query_embedding = embedding_model.get_embeddings(query)[0].values

# cosine similarity is a scikit function
# we are wrapping the query_embedding as a list '[]' becos
# the input is a list of lists
# so_database --> array --> converting into a 2d list
cos_sim_array= cosine_similarity([query_embedding],
                                 list(so_database.embeddings.values))

# other metrics
# distances_argmin instead of cosine_similarity
# index_doc_distances = distances_argmin([query_embedding],
                                      #  list(so_database.embeddings.values))[0]

cos_sim_array # (1, 2000) 1 row 2000 col becos of 1 query

index_doc_cosine = np.argmax(cos_sim_array)

so_database.input_text[index_doc_cosine]

# let us also look at output
so_database.output_text[index_doc_cosine]



The output formatting is not user-friendly. To create a better formatting. Let us use LLM to format the data

In [ ]:
context = "Question: " + so_database.input_text[index_doc_cosine] + \
"\n Answer: " + so_database.output_text[index_doc_cosine]"

In [ ]:

# How to embed and add new embeddings

emb = model.get_embeddings([input_text])[0].values

embeddings_l = question_embeddings.tolist()
# add the new embedding

embeddings_array = np.array(embeddings_l)

print("Shape: " + str(embeddings_array.shape))
print(embeddings_array)

# add the new data to the existing df

new_row = pd.Series([input_text, None, "baking"], index=so.df.columns)
so_df.loc[len(so_df)+1] = new_row

# Classification Example

In [ ]:
clf = IsolationForest(contaimination = 0.005, random_state = 2)

# to view the outliers
preds = clf.fit_predict(embeddings_array)

# drop the newly addded row
so_df = so_df.drop(so_df.index[-1])

# classification
# using emb vectors --> str data
# use it for classification

X = question_embeddings
y = so_df['category'].values

X_train, X_test, y_train, y_test = train_test_split(X,
y, test_size = 0.2, random_state = 2)

clf = RandomForestClassifer(n_estimators = 200)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


# Generate Text Application

In [ ]:
# Generate Text
from vertexai.language_models import TextGenerationModel

generation_Model = TextGenerationModel.from_pretrained("text-bison@001")

prompt = "Recommend me a programming activity to improve my skills"

generation_model.predict(prompt = prompt)

# Decoding Strategies

text = "The garden was full of beautiful"

We can create a probabilities of the next text or tokens (ex [flowers (0.5), trees (0.23), herbs(0.05), ... bugs (0.03)]

probabilities --> Decoding strategy

* Greedy Decoding; The one with highest prob.
* Random sample: Use the probabilities sample a random token
* One of the parameters we can use is called temperature (controls the randomness); low temp are good for good prediction and high temp is good for openended conversation

In Neural Network we wil have outputs called logits. We pass these logits to Softmax to get probability distribution over classes.

Usually (z_i) is the input to softmax. We can divide z_i/$\theta$ (temperature is $\theta$) and pass it into softmax

```
 word  Logits Softmax Softmax with Temperature
                         0.1.     higher
 --------------------------------------------
 flowers.  20  0.881.   1.0        0.28
 trees     18  0.119.   0.00       0.20
 herbs      5  0.000    0.00       0.15
 bugs       2. 0.000.   0.00       0.10
 ```

In [ ]:
prompt = "I reached into my toolkit to fetch my:"

response = generation_model.predict(prompt = prompt, temperature = temperature, )

print(f"[Temperature = {temperature}]")
print(response.text)


Temperature of 0: Deterministic choice; Not creative but reliable

Temperature is increased: Randomness is increased; creative but sometimes unreasonable

# high temp

temperature = 1.0

response = generation_model.predict(prompt = prompt, temperature = temperature, )

print(f"[Temperature = {temperature}]")
print(response.text)



# Different strategies:

# top k
Another Hyp Param is top-k, this means picking top-k probabilities instead of top-1

# top p
Sample from min set of tokens whose cumulative prob is >= p

## ex
(ex [flowers (0.5), trees (0.23), herbs(0.05), ... bugs (0.03)]

top-p = 0.75; then flowers, trees and herbes will be selected.

We dont have to select one strategy

# Putting it all to-gether

* Top K is done first
* from the results top-P is filtered
* Finally Temperature is applied

# Example
* Input Text: The garden was full of beautiful

```[flowers, trees,herbs, plants, ..., weeds, bugs]```

Filters applied in the following order:
* Top K [flowers, trees, hers, plants]
* Top P [flowers, trees]
* Temperature [trees]

* Output: trees

# Example

In [ ]:
prompt = "Write an advertisement for jackets that involve blue elephants and avacados"

top_p = 0.7 # 0 to 1
top_k = 20  # 0 to 40

response = generation_model.predict(prompt = prompt,
                                    temperature = 0.9,
                                    top_k=top_k,
                                    top_p = top_p)

print(f"[Temperature = {temperature}]")
print(top_p)
print(response.text)
